In [177]:
import urllib
import requests
from bs4 import BeautifulSoup
import spacy
from spacy.matcher import PhraseMatcher

def get_lyrics(artist: str, song_title: str):
    """
    Takes in artist name (with space if multiple words) and song title and
    returns the lyrics as a spaCy doc object.
    Can only pull lyrics if found on Genius at the URL formatted as follows:
    genius.com/artist-name-song-name-lyrics
    """    
    
    # Make artist and song names genius URL-friendly
    url_artist = artist.replace(' ','-')
    url_artist = url_artist.replace("'",'')
    url_song_title = song_title.replace(' ','-')
    url_song_title = url_song_title.replace("'",'')
    
    # Make URL and make sure it exists
    url = 'https://genius.com/' + url_artist + '-' + \
        url_song_title + '-lyrics'
    request = requests.get(url)
    if request.status_code != 200:
        return 'Sorry, URL does not exist.'
    
    # Scrape and parse
    req = urllib.request.Request(url, headers={'User-Agent':"Magic Browser"})
    response = urllib.request.urlopen(req)
    html = response.read()
    soup = BeautifulSoup(html, 'html.parser')
    s = soup.text

    # Instantiate spaCy doc and pass in scraped string
    nlp = spacy.load('en_core_web_lg')
    doc = nlp(s)

    # The lyrics start with the song title, then the word "Lyrics". These docs
    # always have that three times, and the second one is the lyrics.  They all
    # end right before "More on Genius".
    
    # Instantiate a spaCy PhraseMatcher
    matcher = PhraseMatcher(nlp.vocab)
    # Add start and end criteria to matcher
    matcher.add("START", [nlp(f'{song_title} ' + 'Lyrics')])
    matcher.add("END", [nlp("More on Genius")])
    matches = matcher(doc)
    if len(matches) == 0:
        return "Couldn't find start"
    # Grab indices from the tuples in matches
    start_index = matches[1][1]
    end_index = matches[2][1] - 1
    lyrics_doc = doc[start_index: end_index]
    
    return lyrics_doc

In [180]:
get_lyrics('Not An Artist','Not a Song')

'Sorry, URL does not exist.'

In [179]:
get_lyrics('John Denver','Country Roads')

Country Roads Lyrics



[Verse 1]
Almost Heaven, West Virginia
Blue Ridge Mountains, Shenandoah River
Life is old there, older than the trees
Younger than the mountains, growing like a breeze

[Chorus]
Country roads, take me home
To the place I belong
West Virginia, mountain mama
Take me home, country roads

[Verse 2]
All my memories gather 'round her
Miner's lady, stranger to blue water
Dark and dusty, painted on the sky
Misty taste of moonshine, teardrop in my eye

[Chorus]
Country roads, take me home
To the place I belong
West Virginia, mountain mama
Take me home, country roads

[Bridge]
I hear her voice in the morning hour, she calls me
The radio reminds me of my home far away
Driving down the road, I get a feeling
That I should have been home yesterday, yesterday

[Chorus]
Country roads, take me home
To the place I belong
West Virginia, mountain mama
Take me home, country roads
Country roads, take me home
To the place I belong
West Virginia, mountain mama
Take me home, country roa

In [186]:
get_lyrics('Cardi B', 'I Like It')

I Like It Lyrics



[Intro]
Yeah, baby, I like it like that
You gotta believe me when I tell you
I said I like it like that
You gotta believe me when I tell you
I said I like it like—

[Verse 1: Cardi B]
Now I like dollars, I like diamonds
I like stuntin', I like shinin' (Yeah)
I like million dollar deals
Where's my pen? Bitch I'm signin' (Signin')
I like those Balenciagas (Those)
The ones that look like socks
I like going to the jeweler
I put rocks all in my watch (Cha-ching)
I like texts from my exes when they want a second chance (What?)
I like proving niggas wrong
I do what they say I can't
They call me Cardi Bardi, banging body
Spicy mami, hot tamale
Hotter than a Somali, fur coat, Ferrari (Rrr, woo)
Hop out the stu', jump in the coupe (Coupe)
Big Dipper on top of the roof
Flexing on bitches as hard as I can
Eating halal, driving the Lam'
Told that bitch I'm sorry though (Sorry though)
'Bout my coins like Mario (Mario)
Yeah, they call me Cardi B
I run this shit like cardio
Woo, fa